# 检查准确率

# 1、输入样点数据进行采样


In [ ]:
import os
import pandas as pd
import numpy as np
import shutil
import arcpy
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error,mean_squared_log_error
import matplotlib.pyplot as plt
import seaborn as sns
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *

In [ ]:
# 分析函数
# 取消并行处理
def disable_parallel_processing(func):
    def wrapper(*args, **kwargs):
        with arcpy.EnvManager(parallelProcessingFactor="0"):
            return func(*args, **kwargs)
    return wrapper
# 采样
def sample_point(point_,raster_,out_name):
    """根据栅格采样点,输出为表格"""
    Sample(raster_,point_,out_name,"NEAREST", "OBJECTID", "CURRENT_SLICE", None, '', None, None, "ROW_WISE", "TABLE")
    return None

# 导出CSV
def export_csv(table_,out_path,out_name):
    TableToTable(table_,out_path,out_name)
    return None

# xy表转点
def xy_to_point(in_table,out_feature_class,x_field,y_field,z_field=None,coordinate_system=arcpy.SpatialReference(4326)):
    """将xy表转点，坐标系默认为WGS84"""
    arcpy.management.XYTableToPoint(in_table=in_table, out_feature_class=out_feature_class, x_field=x_field, y_field=y_field, z_field=z_field, coordinate_system=coordinate_system)
    return None


In [ ]:
# 表格路径
point_data_table = r"C:\Users\Runker\Desktop\dy_result.csv"

In [ ]:
# 点位数据路径
check_point_data_path = r'D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features_dy\pro_check_point'
# 预测结果属性图路径
check_raster_data_path =r'D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb\PH'
# 存储采样数据表的文件地理数据库
sample_gdb_path = r"F:\cache_data\process_table_database\tabledata.gdb" # 需要改变

In [ ]:
# xy表转点
xy_to_point(point_data_table,check_point_data_path,'cyjd','cywd')

In [ ]:
# 采样点数据名称
sample_name = 'pro_check_point'
# 属性图层名称
feature_name = 'PH'
filed_list = [_.name for _ in arcpy.ListFields(check_point_data_path)]
print(filed_list)
# 需要保留的字段
elements_yes = ['OBJECTID', 'ph']
filter_list = [_ for _ in filed_list if _ in elements_yes]
print(filter_list)

In [ ]:
# 使用Delete_management函数删除数据库中的所有内容
try:
    arcpy.Delete_management(sample_gdb_path)
except:
    pass
# 再创建一个新的数据库
arcpy.management.CreateFileGDB(os.path.dirname(sample_gdb_path), "tabledata", "CURRENT")
# 采样并保存到csv文件
sample_point(check_point_data_path,check_raster_data_path,os.path.join(sample_gdb_path,feature_name))

In [ ]:
# 读取原表
old_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(check_point_data_path, elements_yes))

In [ ]:
# 读取采样表
sample_df = pd.DataFrame(arcpy.da.TableToNumPyArray(os.path.join(sample_gdb_path,feature_name), "*", null_value=0))  # 确保数据表中无空值

In [ ]:
# 连接表

result_df = pd.merge(old_df, sample_df, on='OBJECTID', how='left')

In [ ]:

# 评估指标
pre_value = result_df['PH_Band_1']
actual_value = result_df['ph']
# 计算R2
print(f'r2:{r2_score(actual_value, pre_value)}')
# 计算MSE
print(f'mse:{mean_squared_error(actual_value,pre_value)}')
# 计算MAE
print(f'mae:{mean_absolute_error(actual_value,pre_value)}')
# 计算RMSE
print(f'rmse:{np.sqrt(mean_squared_error(actual_value,pre_value))}')
# 计算RMSLE
print(f'rmsle:{np.sqrt(mean_squared_log_error(actual_value,pre_value))}')
# 计算MAPE
print(f'mape:{mean_absolute_percentage_error(actual_value,pre_value)}')


In [ ]:

# 设置Seaborn样式
sns.set(style="whitegrid")

# 创建散点图和回归线
plt.figure(figsize=(10, 5))
sns.regplot(x='ph', y='PH_Band_1', data=result_df[['ph','PH_Band_1']], scatter_kws={'s':50}, line_kws={'color':'red'})

# 添加标题和标签
plt.title('Actual vs Predicted Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

# 显示图表
plt.show()


In [ ]:

# 设置Seaborn样式
sns.set(style="whitegrid")

# 创建图表
plt.figure(figsize=(10, 5))

# 绘制真实值和预测值的散点图
sns.scatterplot(x=result_df.index, y='ph', data=result_df, color='blue', s=100, label='Actual', marker='o')
sns.scatterplot(x=result_df.index, y='PH_Band_1', data=result_df, color='red', s=100, label='Predicted', marker='x')

# 添加回归线
sns.lineplot(x=result_df.index, y='ph', data=result_df, color='blue', linestyle='--')
sns.lineplot(x=result_df.index, y='PH_Band_1', data=result_df, color='red', linestyle='--')

# 添加标题和标签
plt.title('Actual vs Predicted Values')
plt.xlabel('Index')
plt.ylabel('Value')

# 显示图表
plt.legend()
plt.show()
